# Table Extraction with LlamaParse

This notebook will show you how to extract tables and save them as CSV files thanks to LlamaParse advanced parsing capabilities.

Status:
| Last Executed | Version | State      |
|---------------|---------|------------|
| Aug-19-2025   | 0.6.61  | Maintained |

**1. Install needed dependencies**

In [1]:
%pip install llama-cloud-services pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


**2. Set you LLAMA_CLOUD_API_KEY as env variable**

llx-9uejTGZQuoudNZIn4fP67hcRw40XALhIGnZmELPpljSoSQoh

In [2]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-9uejTGZQuoudNZIn4fP67hcRw40XALhIGnZmELPpljSoSQoh"

**3. Initialiaze the parser**

In [3]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    parse_mode="parse_page_with_agent",
    model="openai-gpt-4-1-mini",
    high_res_ocr=True,
    adaptive_long_table=True,
    outlined_table_extraction=True,
    output_tables_as_HTML=True,
)

**4. Get data**

This is a PDF with _lots_ of tables!

In [4]:
! wget https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf

--2025-11-24 10:42:52--  https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf
Resolving assets.accessible-digital-documents.com (assets.accessible-digital-documents.com)... 108.157.254.32, 108.157.254.52, 108.157.254.51, ...
Connecting to assets.accessible-digital-documents.com (assets.accessible-digital-documents.com)|108.157.254.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145494 (142K) [application/pdf]
Saving to: ‘sample-tables.pdf’

sample-tables.pdf   100%[===================>] 142.08K  --.-KB/s    in 0.004s  

2025-11-24 10:42:53 (35.3 MB/s) - ‘sample-tables.pdf’ saved [145494/145494]



**5. Parse document**

In [5]:
result = await parser.aparse("sample-tables.pdf")

Started parsing the file under job_id 028c3931-1871-47cb-b839-e8d6ee87bbc2


**6. Get tables!**

In [6]:
tables = []
for page in result.pages:
    for item in page.items:
        if item.type == "table":
            tables.append(item.rows)

print(tables[8])

[['Rainfall (inches)', 'Americas', 'Asia', 'Europe', 'Africa'], ['Average', '133', '244', '155', '166'], ['24 hour high', '27', '28', '29', '20'], ['12 hour high', '11', '12', '13', '16']]


**7. Load tables**

Let's show one example table!

In [9]:
import pandas as pd
from IPython.display import display

df = pd.DataFrame(tables[8])
df.head()

,0,1,2,3,4
0,Rainfall (inches),Americas,Asia,Europe,Africa
1,Average,133,244,155,166
2,24 hour high,27,28,29,20
3,12 hour high,11,12,13,16


# Task
Process and save all extracted tables from the `tables` list into individual CSV files. The first row of each table will be treated as the header. Then, display a list of the generated CSV file names.

## Process and Save All Tables

### Subtask:
Iterate through the 'tables' list, convert each table (list of lists) into a pandas DataFrame, assuming the first row is the header. Each DataFrame will then be saved to a uniquely named CSV file (e.g., 'table_0.csv', 'table_1.csv').


**Reasoning**:
I need to iterate through the 'tables' list, convert each sub-list into a pandas DataFrame, set the first row as header, remove the header row from the data, save each DataFrame as a uniquely named CSV file, and store the filenames.



In [10]:
csv_filenames = []

for i, table_data in enumerate(tables):
    if not table_data:
        print(f"Skipping empty table at index {i}")
        continue

    # Convert the list of lists to a DataFrame
    df = pd.DataFrame(table_data)

    # Set the first row as header and then drop it from the data
    new_columns = df.iloc[0]
    df = df[1:]
    df.columns = new_columns

    # Construct a unique filename
    filename = f"table_{i}.csv"

    # Save the DataFrame to a CSV file without the index
    df.to_csv(filename, index=False)

    # Append the filename to the list
    csv_filenames.append(filename)

print(f"Generated CSV files: {csv_filenames}")

Generated CSV files: ['table_0.csv', 'table_1.csv', 'table_2.csv', 'table_3.csv', 'table_4.csv', 'table_5.csv', 'table_6.csv', 'table_7.csv', 'table_8.csv', 'table_9.csv', 'table_10.csv', 'table_11.csv', 'table_12.csv', 'table_13.csv', 'table_14.csv', 'table_15.csv', 'table_16.csv', 'table_17.csv', 'table_18.csv', 'table_19.csv', 'table_20.csv', 'table_21.csv', 'table_22.csv', 'table_23.csv', 'table_24.csv', 'table_25.csv', 'table_26.csv', 'table_27.csv', 'table_28.csv']


# Task
Load 'table_8.csv' into a pandas DataFrame, perform a query to find the difference between 'Asia' and 'Americas' average rainfall, and then summarize the querying capabilities of pandas DataFrames for extracted tables.

## Load a Table for Querying

### Subtask:
Load 'table_8.csv' into a pandas DataFrame.


**Reasoning**:
To load the CSV file into a pandas DataFrame, I need to use the pandas library's `read_csv` function. I'll also explicitly import pandas for clarity, although it might have been imported earlier.



In [12]:
import pandas as pd

df_table_8 = pd.read_csv('table_8.csv')
df_table_8.head()

,Rainfall (inches),Americas,Asia,Europe,Africa
0,Average,133,244,155,166
1,24 hour high,27,28,29,20
2,12 hour high,11,12,13,16


## Perform a Complex SQL-like Query

### Subtask:
Demonstrate a complex query on the loaded DataFrame. This could involve filtering rows based on multiple conditions, selecting specific columns, and performing aggregations (e.g., finding the average rainfall for a specific region). Print the results of this query.


**Reasoning**:
I need to access the 'Average' rainfall data for 'Americas' and 'Asia' from the `df_table_8` DataFrame, extract the numerical values, calculate the difference, and then print the result.



In [13]:
average_rainfall = df_table_8[df_table_8['Rainfall (inches)'] == 'Average']

americas_average = average_rainfall['Americas'].values[0]
asia_average = average_rainfall['Asia'].values[0]

difference = asia_average - americas_average

print(f"Difference in average rainfall (Asia - Americas): {difference}")

Difference in average rainfall (Asia - Americas): 111


## Initialize SQLite Database

### Subtask:
Set up an SQLite database connection to prepare for storing the tables.


**Reasoning**:
To set up an SQLite database, I need to import the `sqlite3` module, establish a connection to the database file, and then create a cursor object for executing SQL commands.



In [17]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('tables_data.db')

# Create a cursor object
cursor = conn.cursor()

print("SQLite database 'tables_data.db' connected and cursor created.")

SQLite database 'tables_data.db' connected and cursor created.


In [20]:
def make_unique_columns(columns):
    seen = {}
    unique_cols = []
    for idx, col in enumerate(columns):
        temp_col = str(col).strip() # strip whitespace
        if not temp_col: # if column name is empty after stripping
            temp_col = f"Unnamed_{idx}"

        if temp_col in seen:
            seen[temp_col] += 1
            unique_cols.append(f"{temp_col}_{seen[temp_col]}")
        else:
            seen[temp_col] = 0
            unique_cols.append(temp_col)
    return unique_cols

conn = sqlite3.connect('tables_data.db') # Re-establish connection as it was closed

db_table_names = []

for i, table_data in enumerate(tables):
    if not table_data:
        print(f"Skipping empty table at index {i}")
        continue

    # Convert the list of lists to a DataFrame
    df = pd.DataFrame(table_data)

    # Set the first row as header and then drop it from the data
    # Check if df has enough rows to set a header
    if len(df) > 1:
        # Get potential column names from the first row
        potential_columns = df.iloc[0].tolist()
        # Make column names unique and non-empty
        unique_potential_columns = make_unique_columns(potential_columns)
        df.columns = unique_potential_columns
        df = df[1:].reset_index(drop=True)
    elif len(df) == 1: # If only one row, it becomes the header and the DataFrame will be empty
        potential_columns = df.iloc[0].tolist()
        unique_potential_columns = make_unique_columns(potential_columns)
        df.columns = unique_potential_columns
        df = pd.DataFrame(columns=df.columns)
    else: # Should be caught by `if not table_data` but as a safeguard
        print(f"Table {i} has no data rows after header extraction. Skipping.")
        continue

    # Generate a unique table name
    table_name = f"table_{i}"

    # Write the DataFrame to SQLite
    df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
    db_table_names.append(table_name)

print(f"All tables loaded into SQLite database: {db_table_names}")

# Close the database connection
conn.close()
print("Database connection closed.")

All tables loaded into SQLite database: ['table_0', 'table_1', 'table_2', 'table_3', 'table_4', 'table_5', 'table_6', 'table_7', 'table_8', 'table_9', 'table_10', 'table_11', 'table_12', 'table_13', 'table_14', 'table_15', 'table_16', 'table_17', 'table_18', 'table_19', 'table_20', 'table_21', 'table_22', 'table_23', 'table_24', 'table_25', 'table_26', 'table_27', 'table_28']
Database connection closed.


In [21]:
import sqlite3

# Reconnect to the SQLite database
conn = sqlite3.connect('tables_data.db')
cursor = conn.cursor()

# SQL query to find the average rainfall for Americas and Asia from table_8
query = """
SELECT
    Asia - Americas AS Difference
FROM
    table_8
WHERE
    "Rainfall (inches)" = 'Average'
"""

# Execute the query
cursor.execute(query)

# Fetch the result
difference_result = cursor.fetchone()

if difference_result:
    print(f"Difference in average rainfall (Asia - Americas) from SQLite (table_8): {difference_result[0]}")
else:
    print("Could not find average rainfall data for Americas and Asia in table_8.")

# Close the database connection
conn.close()
print("Database connection closed.")

Difference in average rainfall (Asia - Americas) from SQLite (table_8): 111
Database connection closed.
